## Nhập các thư viện cần thiết

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from RAG import RAGPipeline
import os
from dotenv import load_dotenv
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
import numpy as np
import warnings
import pandas as pd

# Cài đặt bộ dữ liệu ngôn ngữ cho nltk (nếu chưa cài)
# nltk.download('wordnet')
# nltk.download('punkt')
# nltk.download('punkt_tab')


## Đọc dữ liệu từ file Excel

In [3]:
# Đọc file Excel
file_path = 'doc/EvaluateData_RAG_QA.xlsx'  # Đường dẫn đến file Excel của bạn

# Đảm bảo đọc đúng định dạng
try:
    data = pd.read_excel(file_path, engine='openpyxl')
    questions = data['QUESTION'].tolist()
    correct_answers = data['ANSWER'].tolist()
except Exception as e:
    print(f"Lỗi khi đọc file Excel: {e}")

## Sinh ra câu trả lời từ mô hình RAG

In [4]:
from RAG import DocumentStore
from ChunkDocument import DocumentProcessor
from WordReader import WordReader


# Hàm để sinh ra câu trả lời từ mô hình RAG
def generate_answer(question):
    load_dotenv()
    
    # Đọc tài liệu và chuyển đổi chúng thành danh sách các phần văn bản
    file_paths = ["doc/So tay SV HCMUTE-2018.docx"]
    word_reader = WordReader(file_paths)

    # Đọc và xử lý các tài liệu từ WordReader
    sections = word_reader.process_documents()

    # Tạo DocumentProcessor và xử lý các tài liệu
    processor = DocumentProcessor(sections)
    documents = processor.process_documents()

    # Khởi tạo DocumentStore
    store = DocumentStore(documents)

    # API Key của OpenAI (thay thế bằng khóa API của bạn)
    OPENAI_KEY = os.getenv('OPENAI_KEY')
    api_key = OPENAI_KEY

    # Tạo một đối tượng RAGPipeline
    rag_pipeline = RAGPipeline(documents=store, api_key=api_key, model_name='gpt-3.5-turbo')

    # Sinh câu trả lời bằng mô hình RAG
    answer = rag_pipeline.answer_question(question)
    return answer

In [5]:
def compute_similarity(answer1, answer2):
    vectorizer = TfidfVectorizer().fit_transform([answer1, answer2])
    vectors = vectorizer.toarray()
    return cosine_similarity(vectors)[0][1]

## Tính ROUGE và METEOR

METEOR (Metric for Evaluation of Translation with Explicit ORdering):

- Ý Nghĩa: METEOR đo lường sự chính xác ngữ nghĩa của câu trả lời. Nó tính đến đồng nghĩa, hình thức từ và thứ tự từ trong câu.
- Sử Dụng: Khi bạn muốn kiểm tra xem câu trả lời sinh ra có ý nghĩa tương tự như câu trả lời đúng hay không, ngay cả khi cấu trúc từ ngữ không giống nhau.


ROUGE (Recall-Oriented Understudy for Gisting Evaluation):

- Ý Nghĩa: ROUGE thường được sử dụng trong đánh giá tóm tắt văn bản và đo lường mức độ tương đồng giữa các câu bằng cách so sánh các từ và cụm từ.
- Các Chỉ Số Chính:
    - ROUGE-1: Tính số lượng các từ trùng khớp (unigrams) giữa câu sinh ra và câu đúng.
    - ROUGE-L: Đánh giá dựa trên chuỗi con dài nhất (longest common subsequence), cho biết mối quan hệ thứ tự của các từ trong hai câu.

In [6]:
# Hàm tính METEOR
def compute_meteor(generated_answer, correct_answer):
    # Phân tách câu thành danh sách các từ
    generated_tokens = nltk.word_tokenize(generated_answer)
    correct_tokens = nltk.word_tokenize(correct_answer)
    return meteor_score([correct_tokens], generated_tokens)

# Hàm tính ROUGE
def compute_rouge(generated_answer, correct_answer):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(generated_answer, correct_answer)
    return scores['rouge1'].fmeasure, scores['rougeL'].fmeasure  # Sử dụng thuộc tính fmeasure

# Tính METEOR và ROUGE
def calculate_metrics(questions, correct_answers, file_path):
    warnings.filterwarnings("ignore", category=FutureWarning)
    meteor_scores = []
    rouge_scores = []
    rag_answers = []  # Danh sách để lưu câu trả lời của RAG

    for question, correct_answer in zip(questions, correct_answers):
        generated_answer = generate_answer(question)  # Hàm sinh câu trả lời từ mô hình RAG
        rag_answers.append(generated_answer)  # Lưu câu trả lời RAG
        if generated_answer is None:
            continue  # Bỏ qua nếu không thể sinh ra câu trả lời

        # Tính điểm METEOR
        meteor_similarity = compute_meteor(generated_answer, correct_answer)
        meteor_scores.append(meteor_similarity)

        # Tính điểm ROUGE
        rouge_f1_scores = compute_rouge(generated_answer, correct_answer)
        rouge_scores.append(rouge_f1_scores)

    average_meteor = np.mean(meteor_scores) if meteor_scores else 0
    average_rouge_1 = np.mean([score[0] for score in rouge_scores]) if rouge_scores else 0
    average_rouge_l = np.mean([score[1] for score in rouge_scores]) if rouge_scores else 0

    # Đọc DataFrame và cập nhật với câu trả lời của RAG
    data = pd.read_excel(file_path)
    data['RAG_ANSWER'] = rag_answers

    # Lưu DataFrame trở lại file Excel
    data.to_excel(file_path, index=False, engine='openpyxl')

    return average_meteor, (average_rouge_1, average_rouge_l)

In [ ]:
# Gọi hàm để tính METEOR và ROUGE
average_meteor, average_rouge = calculate_metrics(questions, correct_answers, file_path)

# In kết quả
print(f"Average METEOR: {average_meteor}")
print(f"Average ROUGE-1: {average_rouge[0]}")
print(f"Average ROUGE-L: {average_rouge[1]}")